In [1]:
def cosmotensors(gedd):

    geuu = gedd.inverse()
    chris = M.tensor_field(1,2, name='chris')
    for i in range(0,4):
        for j in range(0,4):
            for k in range(0,4):
                chris[i,j,k] = 0.5*sum(geuu[i,dd]*(derivative(gdd[dd,j],coords[k])
                                                  + derivative(gdd[dd,k],coords[j])
                                                  - derivative(gdd[j,k],coords[dd])) for dd in range(0,4))

    riem = M.tensor_field(1,3, name='riem')
    for i in range(0,4):
        for j in range(0,4):
            for k in range(0,4):
                for l in range(0,4):
                    riem[i,j,k,l] = (derivative(chris[i,j,l],coords[k])
                                      - derivative(chris[i,j,k],coords[l])
                                      + sum(chris[i,k,dd]*chris[dd,l,j]
                                          - chris[i,l,dd]*chris[dd,k,j] for dd in range(0,4)))

    ricci = M.tensor_field(0,2, name='ricci')
    for i in range(0,4):
        for k in range(0,4):
            ricci[i,k] = sum(riem[dd,i,dd,k] for dd in range(0,4))

    curv = sum(sum(ricci[i,j]*geuu[i,j] for i in range(0,4)) for j in range(0,4))

    eins = matrix(SR, 4, 4)
    eins = ricci - 0.5*gedd*curv

    return {'inverse_metric': geuu,
            'christoffel': simplify(chris),
            'riemann': simplify(riem),
            'ricci': simplify(ricci),
            'curvature': simplify(curv),
            'einstein': simplify(eins)}

In [2]:
%display latex

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os

In [4]:
#Define spache-time manifold and coordinates

n = 3
M = Manifold(1+n, 'M', structure='Lorentzian')
X.<t,r,th,ph> = M.chart(r"t r:(0,+oo) th:(0,pi):\theta ph:(0,2*pi):\phi")
coords = (t, r, th, ph)

In [5]:
eps = var('eps', latex_name='\\varepsilon')

h = function('h')(r,th)
k = function('k')(r,th)
m = function('m')(r,th)
omega = function('omega', latex_name="\\omega")(r,th)

nu = function('nu', latex_name='\\nu')(r)
lamb = function('lamb', latex_name='\\lambda')(r)

Omegaevalue = function('Omegaevalue', latex_name = "\\Omega_e")(r)
Nevalue = function('Nevalue', latex_name = "N_e")(r)

In [6]:
gdd = M.tensor_field(0,2, name='gdd')
gdd[0,0] = -exp(nu)
gdd[1,1] = exp(lamb)
gdd[2,2] = r^2
gdd[3,3] = r^2*sin(th)^2

K1 = M.tensor_field(0,2, name='K1')
K1[0,3] = omega*r^2*sin(th)^2
K1[3,0] = omega*r^2*sin(th)^2

K2 = M.tensor_field(0,2, name='K2')
K2[0,0] = -4*exp(nu)*h + 2*r^2*sin(th)^2*omega^2
K2[1,1] = 4*exp(lamb)*m
K2[2,2] = 4*k*r^2
K2[3,3] = 4*k*r^2*sin(th)^2

gedd = M.metric()
for i in range(0,n+1):
    for j in range(0,n+1):
        gedd[i,j] = gdd[i,j] + eps*K1[i,j] + 1/2*eps^2*K2[i,j]

tensors = cosmotensors(gedd)
geuu = tensors['inverse_metric']
chris = tensors['christoffel']
riem = tensors['riemann']
ricci = tensors['ricci']
curv = tensors['curvature']
eins = tensors['einstein']

In [7]:
geuu[:]

[                                                                               (2*eps^2*k(r, th) + 1)/(2*eps^4*r^2*k(r, th)*omega(r, th)^2*sin(th)^2 - 2*eps^2*e^nu(r)*h(r, th) - 2*(2*eps^4*e^nu(r)*h(r, th) + eps^2*e^nu(r))*k(r, th) - e^nu(r))                                                                                                                                                                                                                                                  0                                                                                                                                                                                                                                                  0                                                                                     -eps*omega(r, th)/(2*eps^4*r^2*k(r, th)*omega(r, th)^2*sin(th)^2 - 2*eps^2*e^nu(r)*h(r, th) - 2*(2*eps^4*e^nu(r)*h(r, th) + eps^2*e^nu(r))*k(r, th) - e^nu(r))]
[                                                                                                                                                                                                                                                 0                                                                                                                                                                                                         1/(2*eps^2*e^lamb(r)*m(r, th) + e^lamb(r))                                                                                                                                                                                                                                                  0                                                                                                                                                                                                                                                  0]
[                                                                                                                                                                                                                                                 0                                                                                                                                                                                                                                                  0                                                                                                                                                                                                                     1/(2*eps^2*r^2*k(r, th) + r^2)                                                                                                                                                                                                                                                  0]
[                                                                                    -eps*omega(r, th)/(2*eps^4*r^2*k(r, th)*omega(r, th)^2*sin(th)^2 - 2*eps^2*e^nu(r)*h(r, th) - 2*(2*eps^4*e^nu(r)*h(r, th) + eps^2*e^nu(r))*k(r, th) - e^nu(r))                                                                                                                                                                                                                                                  0                                                                                                                                                                                                                                                  0 (eps^2*r^2*omega(r, th)^2*sin(th)^2 - 2*eps^2*e^nu(r)*h(r, th) - e^nu(r))/(2*eps^4*r^4*k(r, th)*omega(r, th)^2*sin(th)^4 - (2*eps^2*r^2*e^nu(r)*h(r, th) + r^2*e^nu(r) + 2*(2*eps^4*r^2*e^nu(r)*h(r, th) + eps^2*r^2*e^nu(r))*k(r, th))*sin(th)^2)]

In [8]:
chris[:]

[[[0,
   -0.500000000000000*(2*eps^2*e^nu(r)*k(r, th) + e^nu(r))*d(nu)/dr/(2*eps^4*r^2*k(r, th)*omega(r, th)^2*sin(th)^2 - 2*eps^2*e^nu(r)*h(r, th) - 2*(2*eps^4*e^nu(r)*h(r, th) + eps^2*e^nu(r))*k(r, th) - e^nu(r)),
   0,
   0],
  [-0.500000000000000*(2*eps^2*e^nu(r)*k(r, th) + e^nu(r))*d(nu)/dr/(2*eps^4*r^2*k(r, th)*omega(r, th)^2*sin(th)^2 - 2*eps^2*e^nu(r)*h(r, th) - 2*(2*eps^4*e^nu(r)*h(r, th) + eps^2*e^nu(r))*k(r, th) - e^nu(r)),
   0,
   0,
   -1.00000000000000*eps*r*omega(r, th)*sin(th)^2/(2*eps^4*r^2*k(r, th)*omega(r, th)^2*sin(th)^2 - 2*eps^2*e^nu(r)*h(r, th) - 2*(2*eps^4*e^nu(r)*h(r, th) + eps^2*e^nu(r))*k(r, th) - e^nu(r))],
  [0,
   0,
   0,
   -1.00000000000000*eps*r^2*cos(th)*omega(r, th)*sin(th)/(2*eps^4*r^2*k(r, th)*omega(r, th)^2*sin(th)^2 - 2*eps^2*e^nu(r)*h(r, th) - 2*(2*eps^4*e^nu(r)*h(r, th) + eps^2*e^nu(r))*k(r, th) - e^nu(r))],
  [0,
   -1.00000000000000*eps*r*omega(r, th)*sin(th)^2/(2*eps^4*r^2*k(r, th)*omega(r, th)^2*sin(th)^2 - 2*eps^2*e^nu(r)*h(r, th) - 2*(2*eps^4*e^nu(r)*h(r, th) + eps^2*e^nu(r))*k(r, th) - e^nu(r)),
   -1.00000000000000*eps*r^2*cos(th)*omega(r, th)*sin(th)/(2*eps^4*r^2*k(r, th)*omega(r, th)^2*sin(th)^2 - 2*eps^2*e^nu(r)*h(r, th) - 2*(2*eps^4*e^nu(r)*h(r, th) + eps^2*e^nu(r))*k(r, th) - e^nu(r)),
   0]],
 [[0.500000000000000*e^nu(r)*d(nu)/dr/(2*eps^2*e^lamb(r)*m(r, th) + e^lamb(r)),
   0,
   0,
   0],
  [0, 0.500000000000000*d(lamb)/dr/(2*eps^2*m(r, th) + 1), 0, 0],
  [0, 0, -1.00000000000000*r/(2*eps^2*e^lamb(r)*m(r, th) + e^lamb(r)), 0],
  [0,
   0,
   0,
   -1.00000000000000*r*sin(th)^2/(2*eps^2*e^lamb(r)*m(r, th) + e^lamb(r))]],
 [[0, 0, 0, 0],
  [0, 0, 1.00000000000000/(2*eps^2*r*k(r, th) + r), 0],
  [0, 1.00000000000000/(2*eps^2*r*k(r, th) + r), 0, 0],
  [0, 0, 0, -1.00000000000000*cos(th)*sin(th)/(2*eps^2*k(r, th) + 1)]],
 [[0,
   0.500000000000000*eps*e^nu(r)*omega(r, th)*d(nu)/dr/(2*eps^4*r^2*k(r, th)*omega(r, th)^2*sin(th)^2 - 2*eps^2*e^nu(r)*h(r, th) - 2*(2*eps^4*e^nu(r)*h(r, th) + eps^2*e^nu(r))*k(r, th) - e^nu(r)),
   0,
   0],
  [0.500000000000000*eps*e^nu(r)*omega(r, th)*d(nu)/dr/(2*eps^4*r^2*k(r, th)*omega(r, th)^2*sin(th)^2 - 2*eps^2*e^nu(r)*h(r, th) - 2*(2*eps^4*e^nu(r)*h(r, th) + eps^2*e^nu(r))*k(r, th) - e^nu(r)),
   0,
   0,
   1.00000000000000*(eps^2*r^2*omega(r, th)^2*sin(th)^2 - 2*eps^2*e^nu(r)*h(r, th) - e^nu(r))/(2*eps^4*r^3*k(r, th)*omega(r, th)^2*sin(th)^2 - 2*eps^2*r*e^nu(r)*h(r, th) - r*e^nu(r) - 2*(2*eps^4*r*e^nu(r)*h(r, th) + eps^2*r*e^nu(r))*k(r, th))],
  [0,
   0,
   0,
   1.00000000000000*(eps^2*r^2*cos(th)*omega(r, th)^2*sin(th)^2 - (2*eps^2*e^nu(r)*h(r, th) + e^nu(r))*cos(th))/(2*eps^4*r^2*k(r, th)*omega(r, th)^2*sin(th)^3 - (2*eps^2*e^nu(r)*h(r, th) + 2*(2*eps^4*e^nu(r)*h(r, th) + eps^2*e^nu(r))*k(r, th) + e^nu(r))*sin(th))],
  [0,
   1.00000000000000*(eps^2*r^2*omega(r, th)^2*sin(th)^2 - 2*eps^2*e^nu(r)*h(r, th) - e^nu(r))/(2*eps^4*r^3*k(r, th)*omega(r, th)^2*sin(th)^2 - 2*eps^2*r*e^nu(r)*h(r, th) - r*e^nu(r) - 2*(2*eps^4*r*e^nu(r)*h(r, th) + eps^2*r*e^nu(r))*k(r, th)),
   1.00000000000000*(eps^2*r^2*cos(th)*omega(r, th)^2*sin(th)^2 - (2*eps^2*e^nu(r)*h(r, th) + e^nu(r))*cos(th))/(2*eps^4*r^2*k(r, th)*omega(r, th)^2*sin(th)^3 - (2*eps^2*e^nu(r)*h(r, th) + 2*(2*eps^4*e^nu(r)*h(r, th) + eps^2*e^nu(r))*k(r, th) + e^nu(r))*sin(th)),
   0]]]

In [9]:
riem[:]

[[[[0,
    0,
    0,
    0.5*eps*r*e^nu(r)*omega(r, th)*sin(th)^2*d(nu)/dr/(2*(2*eps^6*r^2*e^lamb(r)*k(r, th)*m(r, th) + eps^4*r^2*e^lamb(r)*k(r, th))*omega(r, th)^2*sin(th)^2 - 2*eps^2*e^(lamb(r) + nu(r))*h(r, th) - 2*(2*eps^4*e^(lamb(r) + nu(r))*h(r, th) + eps^2*e^(lamb(r) + nu(r)))*k(r, th) - 2*(2*eps^4*e^(lamb(r) + nu(r))*h(r, th) + eps^2*e^(lamb(r) + nu(r)) + 2*(2*eps^6*e^(lamb(r) + nu(r))*h(r, th) + eps^4*e^(lamb(r) + nu(r)))*k(r, th))*m(r, th) - e^(lamb(r) + nu(r)))],
   [0,
    0,
    -((4.0*eps^6*r^2*e^nu(r)*k(r, th)^2 + 2.0*eps^4*r^2*e^nu(r)*k(r, th) - 0.5*eps^2*r^2*e^nu(r))*cos(th)*omega(r, th)^2*sin(th)*d(nu)/dr + (1.0*eps^4*r^2*e^nu(r)*omega(r, th)^2*d(k)/dth*d(nu)/dr + (4.0*eps^6*r^2*e^nu(r)*k(r, th)^2 + 2.0*eps^4*r^2*e^nu(r)*k(r, th))*omega(r, th)*d(nu)/dr*d(omega)/dth)*sin(th)^2 + (-4.0*eps^6*e^(2*nu(r))*k(r, th)^2 - 4.0*eps^4*e^(2*nu(r))*k(r, th) - 1.0*eps^2*e^(2*nu(r)))*d(h)/dth*d(nu)/dr)/(4*eps^8*r^4*k(r, th)^2*omega(r, th)^4*sin(th)^4 + 4*eps^4*e^(2*nu(r))*h(r, th)^2 - 4*(2*(2*eps^8*r^2*e^nu(r)*h(r, th) + eps^6*r^2*e^nu(r))*k(r, th)^2 + (2*eps^6*r^2*e^nu(r)*h(r, th) + eps^4*r^2*e^nu(r))*k(r, th))*omega(r, th)^2*sin(th)^2 + 4*eps^2*e^(2*nu(r))*h(r, th) + 4*(4*eps^8*e^(2*nu(r))*h(r, th)^2 + 4*eps^6*e^(2*nu(r))*h(r, th) + eps^4*e^(2*nu(r)))*k(r, th)^2 + 4*(4*eps^6*e^(2*nu(r))*h(r, th)^2 + 4*eps^4*e^(2*nu(r))*h(r, th) + eps^2*e^(2*nu(r)))*k(r, th) + e^(2*nu(r))),
    0],
   [0,
    ((4.0*eps^6*r^2*e^nu(r)*k(r, th)^2 + 2.0*eps^4*r^2*e^nu(r)*k(r, th) - 0.5*eps^2*r^2*e^nu(r))*cos(th)*omega(r, th)^2*sin(th)*d(nu)/dr + (1.0*eps^4*r^2*e^nu(r)*omega(r, th)^2*d(k)/dth*d(nu)/dr + (4.0*eps^6*r^2*e^nu(r)*k(r, th)^2 + 2.0*eps^4*r^2*e^nu(r)*k(r, th))*omega(r, th)*d(nu)/dr*d(omega)/dth)*sin(th)^2 + (-4.0*eps^6*e^(2*nu(r))*k(r, th)^2 - 4.0*eps^4*e^(2*nu(r))*k(r, th) - 1.0*eps^2*e^(2*nu(r)))*d(h)/dth*d(nu)/dr)/(4*eps^8*r^4*k(r, th)^2*omega(r, th)^4*sin(th)^4 + 4*eps^4*e^(2*nu(r))*h(r, th)^2 - 4*(2*(2*eps^8*r^2*e^nu(r)*h(r, th) + eps^6*r^2*e^nu(r))*k(r, th)^2 + (2*eps^6*r^2*e^nu(r)*h(r, th) + eps^4*r^2*e^nu(r))*k(r, th))*omega(r, th)^2*sin(th)^2 + 4*eps^2*e^(2*nu(r))*h(r, th) + 4*(4*eps^8*e^(2*nu(r))*h(r, th)^2 + 4*eps^6*e^(2*nu(r))*h(r, th) + eps^4*e^(2*nu(r)))*k(r, th)^2 + 4*(4*eps^6*e^(2*nu(r))*h(r, th)^2 + 4*eps^4*e^(2*nu(r))*h(r, th) + eps^2*e^(2*nu(r)))*k(r, th) + e^(2*nu(r))),
    0,
    0],
   [-0.5*eps*r*e^nu(r)*omega(r, th)*sin(th)^2*d(nu)/dr/(2*(2*eps^6*r^2*e^lamb(r)*k(r, th)*m(r, th) + eps^4*r^2*e^lamb(r)*k(r, th))*omega(r, th)^2*sin(th)^2 - 2*eps^2*e^(lamb(r) + nu(r))*h(r, th) - 2*(2*eps^4*e^(lamb(r) + nu(r))*h(r, th) + eps^2*e^(lamb(r) + nu(r)))*k(r, th) - 2*(2*eps^4*e^(lamb(r) + nu(r))*h(r, th) + eps^2*e^(lamb(r) + nu(r)) + 2*(2*eps^6*e^(lamb(r) + nu(r))*h(r, th) + eps^4*e^(lamb(r) + nu(r)))*k(r, th))*m(r, th) - e^(lamb(r) + nu(r))),
    0,
    0,
    0]],
  [[0,
    -(((2.0*eps^6*r^2*e^nu(r)*m(r, th) + 1.0*eps^4*r^2*e^nu(r))*omega(r, th)^2*d(k)/dr*d(nu)/dr + (-2.0*eps^6*r^2*e^nu(r)*k(r, th)^2 - 1.0*eps^4*r^2*e^nu(r)*k(r, th) + (-4.0*eps^8*r^2*e^nu(r)*k(r, th)^2 - 2.0*eps^6*r^2*e^nu(r)*k(r, th))*m(r, th))*omega(r, th)^2*d(nu)/dr^2 + (-2.0*eps^6*r^2*e^nu(r)*k(r, th)^2 - 1.0*eps^4*r^2*e^nu(r)*k(r, th) + (-4.0*eps^8*r^2*e^nu(r)*k(r, th)^2 - 2.0*eps^6*r^2*e^nu(r)*k(r, th))*m(r, th))*omega(r, th)^2*d^2(nu)/dr^2 + (4.0*eps^6*r^2*e^nu(r)*k(r, th)^2 + 2.0*eps^4*r^2*e^nu(r)*k(r, th) + (8.0*eps^8*r^2*e^nu(r)*k(r, th)^2 + 4.0*eps^6*r^2*e^nu(r)*k(r, th))*m(r, th))*omega(r, th)*d(nu)/dr*d(omega)/dr + ((1.0*eps^6*r^2*e^nu(r)*k(r, th)^2 + 0.5*eps^4*r^2*e^nu(r)*k(r, th))*omega(r, th)^2*d(lamb)/dr + (4.0*eps^6*r*e^nu(r)*k(r, th)^2 + 2.0*eps^4*r*e^nu(r)*k(r, th) - 0.5*eps^2*r*e^nu(r) + (8.0*eps^8*r*e^nu(r)*k(r, th)^2 + 4.0*eps^6*r*e^nu(r)*k(r, th) - 1.0*eps^4*r*e^nu(r))*m(r, th))*omega(r, th)^2)*d(nu)/dr)*sin(th)^2 + (-4.0*eps^6*e^(2*nu(r))*k(r, th)^2 - 4.0*eps^4*e^(2*nu(r))*k(r, th) - 1.0*eps^2*e^(2*nu(r)) + (-8.0*eps^8*e^(2*nu(r))*k(r, th)^2 - 8.0*eps^6*e^(2*nu(r))*k(r, th) - 2.0*eps^4*e^(2*nu(r)))*m(r, th))*d(h)/dr*d(nu)/dr + (

In [10]:
ricci[:]

[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [11]:
curv[:]

TypeError: 'ChartFunctionRing_with_category.element_class' object is not subscriptable

In [ ]:
eins[:]

In [5]:
#Construct the metric

if os.path.exists('metric.sobj')==False:
    
    gdd =  M.tensor_field(0,2, name='gdd')
    gdd[0,0] = -exp(nu)
    gdd[1,1] = exp(lamb)
    gdd[2,2] = r^2
    gdd[3,3] = r^2*sin(th)^2

    K1 = M.tensor_field(0,2, name='K1')
    K1[0,3] = omega*r^2*sin(th)^2
    K1[3,0] = omega*r^2*sin(th)^2

    K2 = M.tensor_field(0,2, name='K2')
    K2[0,0] = -4*exp(nu)*h + 2*r^2*sin(th)^2*omega^2
    K2[1,1] = 4*exp(lamb)*m
    K2[2,2] = 4*k*r^2
    K2[3,3] = 4*k*r^2*sin(th)^2

    gedd = M.metric()
    for i in range(0,n+1):
        for j in range(0,n+1):
            gedd[i,j] = gdd[i,j] + eps*K1[i,j] + 1/2*eps^2*K2[i,j]
    
    geuu = gedd.inverse()

    #all_tensors = {gdd}#, 'gedd': gedd, 'geuu': geuu, 'K1': K1, 'K2': K2}
    save(gdd, 'metric')
    
else:
    
    gdd = load('metric.sobj')
    #gedd = my_metric['gedd']
    #geuu = my_metric['geuu']
    #K1 = my_metric['K1']
    #K2 = my_metric['K2']

NameError: name 'nu' is not defined

In [ ]:
show(LatexExpr("g_{\\varepsilon}\\rvert_{\\varepsilon=0} = "), gdd[:])
show(LatexExpr("K_1 = "), K1[:])
show(LatexExpr("K_2 = "), K2[:])
show(LatexExpr("g_{\\varepsilon} = "), gedd[:])

TypeError: sequence item 0: expected str instance, bytes found

In [7]:
#Construct Einstein's tensor

if os.path.exists('tensors.sobj')==False:
    
    ricci = gedd.ricci()
    s_curv = geuu['^{ab}']*ricci['_ab']
    eins = ricci - gdd*s_curv/2                                                                           #G_\mu\nu e
    
    #deins = M.tensor_field(0,2,'deins')
    #ddeins = M.tensor_field(0,2,'ddeins')
    #for i in range(0,n+1):
    #    for j in range(0,n+1):
    #        deins[i,j] = diff(eins[i,j].expr(),eps)
    #        ddeins[i,j] = diff(eins[i,j].expr(),eps,eps)

    all_tensors = {'eins': eins}#, 'deins': deins, 'ddeins': ddeins}
    save(all_tensors, 'tensors')
    
else:
    
    my_tensors = load('tensors.sobj')
    eins = my_tensors['eins']
    deins = my_tensors['deins']
    ddeins = my_tensors['ddeins']

TypeError: ECL says: THROW: The catch MACSYMA-QUIT is undefined.

In [7]:
show(LatexExpr("g_{\\varepsilon}\\rvert_{\\varepsilon=0} = "), gdd[:])
show(LatexExpr("K_1 = "), K1[:])
show(LatexExpr("K_2 = "), K2[:])
show(LatexExpr("g_{\\varepsilon} = "), gedd[:])

g_{\varepsilon}\rvert_{\varepsilon=0} =  [     -e^nu(r)             0             0             0]
[            0     e^lamb(r)             0             0]
[            0             0           r^2             0]
[            0             0             0 r^2*sin(th)^2]

K_1 =  [                         0                          0                          0 r^2*omega(r, th)*sin(th)^2]
[                         0                          0                          0                          0]
[                         0                          0                          0                          0]
[r^2*omega(r, th)*sin(th)^2                          0                          0                          0]

K_2 =  [2*r^2*omega(r, th)^2*sin(th)^2 - 4*e^nu(r)*h(r, th)                                                   0                                                   0                                                   0]
[                                                  0                                4*e^lamb(r)*m(r, th)                                                   0                                                   0]
[                                                  0                                                   0                                      4*r^2*k(r, th)                                                   0]
[                                                  0                                                   0                                                   0                            4*r^2*k(r, th)*sin(th)^2]

g_{\varepsilon} =  [eps^2*r^2*omega(r, th)^2*sin(th)^2 - 2*eps^2*e^nu(r)*h(r, th) - e^nu(r)                                                                       0                                                                       0                                          eps*r^2*omega(r, th)*sin(th)^2]
[                                                                      0                                  2*eps^2*e^lamb(r)*m(r, th) + e^lamb(r)                                                                       0                                                                       0]
[                                                                      0                                                                       0                                              2*eps^2*r^2*k(r, th) + r^2                                                                       0]
[                                         eps*r^2*omega(r, th)*sin(th)^2                                                                       0                                                                       0                                  (2*eps^2*r^2*k(r, th) + r^2)*sin(th)^2]

In [ ]:
show(LatexExpr("g_{\\varepsilon}\\rvert_{\\varepsilon=0} = "), eins[:])
show(LatexExpr("K_1 = "), K1[:])
show(LatexExpr("K_2 = "), K2[:])
show(LatexExpr("g_{\\varepsilon} = "), gedd[:])

TypeError: sequence item 0: expected str instance, bytes found

In [ ]:
#4-velocities of the fluid for any epsilon

ue = M.vector_field(name="ue")
ue[0] = Ne; ue[1] = 0; ue[2] = 0; ue[3] = Ne*Omegae

u = M.vector_field(name="u")
u[0] = exp(-nu/2); u[1] =  0; u[2] =  0; u[3] =  0

Nevalue = solve(u[0].expr()==ue[0].expr(), Ne)[0].rhs()                                                    #Eq 4.20
Omegaevalue = solve(u[3].expr()==ue[3].expr(), Omegae)[0].rhs()                                            #Eq 4.20

In [ ]:
show(LatexExpr("N_{\\varepsilon}\\rvert_{\\varepsilon=0} = "),Nevalue)
show(LatexExpr("\\Omega_{\\varepsilon}\\rvert_{\\varepsilon=0} = "), Omegaevalue)

In [ ]:
u1 = M.vector_field(name="u1",latex_name="u^{(1)}")
u2 = M.vector_field(name="u2",latex_name="u^{(2)}")

u10 = function('u10',latex_name="u^{(1)0}")(r)
u20 = function('u20',latex_name="u^{(2)0}")(r)

Omega1 = function('Omega1', latex_name="\\Omega^{(1)}")(eps)
Omega2 = function('Omega2', latex_name="\\Omega^{(2)}")(eps)

s0 = eps == 0
s1 = Ne.subs(eps==0) == Nevalue
s2 = diff(Ne,eps).subs(eps==0) == u10
s3 = diff(Omegae,eps).subs(eps==0) == Omega1
s4 = Omegae.subs(eps==0) == Omegaevalue
s5 = diff(Omegae,eps,eps).subs(eps==0) == Omega2
s6 = diff(Ne,eps,eps).subs(eps==0) == u20

sustituciones = (s0,s1,s2,s4,s3,s5,s6)

for i in range(0,n+1):
    u1[i] = diff(ue[i].expr(),eps).subs(sustituciones)
    u2[i] = diff(ue[i].expr(),eps,eps).subs(sustituciones)

Ue = ue.down(gedd)
geUeue = Ue['_a']*ue['^a']
dgeUeue = diff(geUeue.expr(), eps).subs(sustituciones) == 0
u10value = solve(dgeUeue, u10)[0].rhs()
s7 = u10 == u10value
u1.apply_map(lambda cmp: cmp.subs(u10==u10value))                                                          #Eq 4.21

ddgeUeue = diff(geUeue.expr(), eps, eps).subs(sustituciones) == 0
u20value = solve(ddgeUeue, u20)[0].rhs()
s8 = u20 == u20value
u2.apply_map(lambda cmp: cmp.subs(u20==u20value).subs(u10==u10value))                                      #Eq 4.22

In [6]:
show(u1.display())
show(u2.display())

NameError: name 'u1' is not defined

In [ ]:
deins.apply_map(lambda cmp: cmp.subs(sustituciones))                                             #d/de(G_\mu\nu)|e=0
ddeins.apply_map(lambda cmp: cmp.subs(sustituciones))                                        #d^2/de^2(G_\mu\nu)|e=0

In [ ]:
Ee = function('Ee', latex_name="E_{\\varepsilon}")(r,eps)
Pe = function('Pe', latex_name="P_{\\varepsilon}")(r,eps)

eins0 = M.tensor_field(0,2, 'eins0', latex_name='G_0')
Tedd = M.tensor_field(0,2,name='Tedd',latex_name="Tdd_{\\varepsilon}")
Tdd = M.tensor_field(0,2, 'Tdd')
for i in range(0,n+1):
    for j in range(0,n+1):
        eins0[i,j] = eins[i,j]
        Tedd[i,j] = (Ee + Pe)*Ue[i]*Ue[j] + Pe*gedd[i,j]                                                 #T_\mu\nu e
        Tdd[i,j] = Tedd[i,j]

eins0.apply_map(lambda cmp: cmp.subs(sustituciones))                                                       #G_\mu\nu
Tdd.apply_map(lambda cmp: cmp.subs(sustituciones))                                                         #T_\mu\nu

In [ ]:
show(LatexExpr("T_{\\varepsilon}\\rvert_{\\varepsilon=0} = "), Tdd[:])

In [ ]:
kappa = 8*pi
E = function('E')(r)
P = function('P')(r)

s9 = Ee.subs(eps==0) == E
s10 = Pe.subs(eps==0) == P
sustituciones = (s1,s2,s3,s4,s5,s6,s7,s8,s9,s10)
Tdd.apply_map(lambda cmp: cmp.subs(sustituciones))
E_tt = solve(eins0[0,0].expr()==kappa*Tdd[0,0].expr(),E)[0].rhs()                                            #Eq 3.7
P_rr = solve(eins0[1,1].expr()==kappa*Tdd[1,1].expr(),P)[0].rhs()                                            #Eq 3.8
P_thth = solve(eins0[2,2].expr()==kappa*Tdd[2,2].expr(),P)[0].rhs()
P_phph = solve(eins0[3,3].expr()==kappa*Tdd[3,3].expr(),P)[0].rhs()
ddnu = solve(P_rr == P_thth, diff(nu,r,r))[0].rhs()                                                          #Eq 3.9

In [ ]:
show(LatexExpr("E = "), E_tt)
show(LatexExpr("P_{rr} = "), P_rr)
show(LatexExpr("\\nu'' = "), ddnu)

In [ ]:
dlamb = solve(E==E_tt,diff(lamb,r))[0].rhs()                                                                 #Eq 3.10
dnu = solve(P==P_rr,diff(nu,r))[0].rhs()                                                                     #Eq 3.11

In [ ]:
show(LatexExpr("\\lambda' = "), dlamb)
show(LatexExpr("\\nu' = "), dnu)

In [ ]:
E1 = function('E1', latex_name='E^{(1)}')(r)
E2 = function('E2', latex_name='E^{(2)}')(r)
P1 = function('P1', latex_name='P^{(1)}')(r)
P2 = function('P2', latex_name='P^{(2)}')(r)

s11 = diff(Ee,eps).subs(eps==0) == E1
s12 = diff(Ee,eps,eps).subs(eps==0) == E2
s13 = diff(Pe,eps).subs(eps==0) == P1
s14 = diff(Pe,eps,eps).subs(eps==0) == P2

s15 = m == m0 + m2*(3*cos(th)^2-1)/2
s16 = k == k0 + k2*(3*cos(th)^2-1)/2
s17 = h == h0 + h2*(3*cos(th)^2-1)/2

sustituciones = (s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17)

In [ ]:
dTdd = M.tensor_field(0,2,'dTdd')
ddTdd = M.tensor_field(0,2,'ddTdd')
for i in range(0,n+1):
    for j in range(0,n+1):
        dTdd[i,j] = diff(Tedd[i,j].expr(),eps)
        ddTdd[i,j] = diff(Tedd[i,j].expr(),eps,eps)
dTdd.apply_map(lambda cmp: cmp.subs(sustituciones))                                              #d/de(T_\mu\nu)|e=0
ddTdd.apply_map(lambda cmp: cmp.subs(sustituciones))                                         #d^2/de^2(T_\mu\nu)|e=0

In [ ]:
# Calculamos ahora la primera derivada de la ecuación

show(deins[1,1],LatexExpr(" = "), kappa*dTdd[1,1])
show(deins[2,2],LatexExpr(" = "), kappa*dTdd[2,2])
show(deins[3,3],LatexExpr(" = "), kappa*dTdd[3,3])

P1 = solve(deins[1,1].expr() == kappa*dTdd[1,1].expr(),P1)[0].rhs()

show(LatexExpr("\\text{De aquí sacamos }P^{1}(r)\\rightarrow\\text{$P^1(r)$ = }"), P1)

In [ ]:
show(deins[0,0],LatexExpr(" = "), kappa*dTdd[0,0])

E1 = solve(deins[0,0].expr() == kappa*dTdd[0,0].expr(),E1)[0].rhs()
E1 = E1.subs(sustituciones)
show(LatexExpr("\\text{De aquí sacamos }E^{1}(r)\\rightarrow\\text{$E^1(r)$ = }"), E1)

In [ ]:
show(deins[0,3].expr() == kappa*dTdd[0,3].expr())                                                           #Eq 48

In [ ]:
# Para la segunda derivada tenemos

show(ddeins[0,0].expr(),LatexExpr(" = "), kappa*ddTdd[0,0].expr())

In [ ]:
show(ddeins[1,1],LatexExpr(" = "), kappa*ddTdd[1,1])

In [ ]:
show(ddeins[2,2],LatexExpr(" = "), kappa*ddTdd[2,2])

In [ ]:
show(ddeins[3,3],LatexExpr(" = "), kappa*ddTdd[3,3])

In [ ]:
show(ddeins[0,3],LatexExpr(" = "), kappa*ddTdd[0,3])

In [ ]:
show(ddeins[1,2],LatexExpr(" = "), kappa*ddTdd[1,2])

In [ ]:
# Aplicamos barotropic EoS
# Utilizando (6.11) de Reina con P1=0 y E1=0
E2*diff(P,r) - P2*diff(E,r) == 0

In [ ]:
h0 = function('h0', latex_name="h_0")(r,th)
k0 = function('k0', latex_name="k_0")(r,th)
m0 = function('m0', latex_name="m_0")(r,th)

h2 = function('h2', latex_name="h_2")(r,th)
k2 = function('k2', latex_name="k_2")(r,th)
m2 = function('m2', latex_name="m_2")(r,th)

In [ ]:
m = m0 + m2*(3*cos(th)^2-1)/2
k = k0 + k2*(3*cos(th)^2-1)/2
h = h0 + h2*(3*cos(th)^2-1)/2

In [ ]:
ddeins[1,1]